# Gaugefileds.jl で Polyakov ループの計算をしてみる

## SU(N) 格子ゲージ理論

### プラケットゲージ作用

$S_{SU(N)} = \beta\sum_n\sum_{\mu\ne\nu}\frac{1}{2N}\mathrm{tr}\left[1-P_{\mu\nu}(n)\right],$

$\beta \equiv 2N/g^2$ : 格子ゲージ結合定数,

$U_{n,\mu} \in$ SU(N) : リンク変数,

$P_{\mu\nu}(n) \equiv U_\mu(n)U_\nu(n+\hat{\mu})U_\mu^\dagger(n+\hat{\nu})U_\nu^\dagger(n)$ : プラケット.

## Polyakov ループ

$\Phi(\vec{n}) \equiv \mathrm{tr}\prod_{n_4=1}^{N_\tau}U_4(\vec{n},n_4)$

## 数値計算アルゴリズム

### 擬熱浴法

## 数値計算コード

### Gaugefeilds.jl

* ウェブサイト: https://github.com/akio-tomiya/Gaugefields.jl

---

## 数値計算の実例

### ライブラリのインストール

In [ ]:
using Pkg

Pkg.add("Gaugefields")
Pkg.add("Plots")

### ライブラリの読み込み

In [2]:
using Gaugefields # 格子ゲージ理論計算のライブラリ
using Plots       # 描画ライブラリ
using Random      # 乱数ライブラリ
using Statistics  # 統計計算ライブラリ

### 定数の定義
デフォルトのセットアップは計算時間が短くなるよう、体積やサンプリング間隔が小さめ。

色々変えて結果がどう変わるか確認してみよう。

In [ ]:
const nwing = 1      # Gaugefields.jl 特有のパラメータ
const seed = 1234567 # 乱数の種

const nc = 3                      # color の数
const lattice_size = [8, 8, 8, 4] # 格子サイズ (x, y, z, t)

const ntherm = 100                                         # 熱平衡状態までの配位の更新回数 (最終的な計算から除外)
const nsweeps_per_sample = 30                              # 次の配位をサンプルするまでの更新回数
const nsamples = 100                                       # 配位のサンプル数
const nsweep_all = ntherm + nsweeps_per_sample * nsamples  # 総更新回数

const betas = [5.00, 5.25, 5.50, 5.60, 5.65, 5.70, 5.75, 5.80, 5.85, 5.90, 5.95, 6.00]; # 結合定数

const plt1 = plot(
    xlims = (0, 1.35*nsweep_all),
    xlabel="MC steps",
    ylabel="|Φ|",
    legend=:topright)

const plt2 = plot(
        xlims = (-1, 1),
        ylims = (-1, 1),
        xlabel="Re Φ",
        ylabel="Im Φ",
        legend=:topright)

### ゲージ配位の更新と Polyakove ループの計算

In [ ]:
function update!(U, beta)

    temp = Array{typeof(U[1]),1}(undef, 3) # 作業用変数
    for i = 1:3
        temp[i] = similar(U[1])
    end

    polys = ComplexF64[]    # Polyakov ループ保存用変数

    # ゲージ配位の更新
    for _ = 1:nsweep_all

        heatbath!(U, temp, beta) # 擬熱浴法
        poly = calculate_Polyakov_loop(U, temp[1], temp[2]) # Polyakov ループの計算
        push!(polys, poly)

    end

    # Polyakov ループ履歴をファイルに書き出し
    #=
    open("poly_beta$beta.txt", "w") do f
        for i = 1:nsweep_all
            println(f, polys[i])
        end
    end
    =#

    # Polyakov ループ履歴をグラフに出力
    abs_polys = abs.(polys)
    display(plot!(
        plt1,
        abs_polys,
        label="β = $beta"))
    sleep(0.1)

    polys_used = @view polys[ntherm:nsweeps_per_sample:end] # 計算に用いる分のデータを抽出

    # Polyakov ループの分布をグラフに出力
    display(plot!(
        plt2,
        real.(polys_used),
        imag.(polys_used),
        seriestype=:scatter,
        label="β = $beta"))
    sleep(0.1)

    polys_used = @view abs_polys[ntherm:nsweeps_per_sample:end] # 計算に用いる分のデータを抽出
    poly_ave = mean(polys_used)                                 # Polyakov ループ期待値の計算
    poly_err = sqrt(varm(polys_used, poly_ave) / nsamples)      # Polyakov ループの統計誤差の計算

    return poly_ave, poly_err

end

### シミュレーションの実行

In [ ]:
function run()

    # 乱数の種の設定
    Random.seed!(seed)

    # ゲージ配位の初期化
    U = Initialize_Gaugefields(nc, nwing, lattice_size..., condition="hot")

    poly_aves = Float64[]
    poly_errs = Float64[]

    # シミュレーションを実行
    for beta in betas

        poly_ave, poly_err = update!(U, beta)
        push!(poly_aves, poly_ave)
        push!(poly_errs, poly_err)

    end

    # Polyakov ループの結果をファイルに書き出し
    #=
    open("poly.txt", "w") do f

        println(f, "# beta  poly  error")

        for i in eachindex(betas)

            println(f, "$(betas[i]) $(poly_aves[i]) $(poly_errs[i])")

        end

    end
    =#

    # Polyakov ループの結果をグラフに出力
    display(plot(
        betas,
        poly_aves,
        yerror=poly_errs,
        seriestype=:scatter,
        markercolor=:black,
        markershape=:circle,
        label="Heatbath",
        xlabel="β",
        ylabel="<|Φ|>",
        legend=:topleft))

end

In [ ]:
run()